In [1]:
import os
import sys
import numpy as np
import pandas as pd

import rocks
rocks.set_log_level("error")

import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import angular_separation


import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes



In [2]:
# import figure_setup as fs

# Get ZTF fit

In [3]:
# Local Configuration
data_fink = '../'
bft_file = os.path.join(data_fink,'data','ssoBFT-latest.parquet')



In [4]:
# Threshold for selection (of non-zero values)
thres = 1e-3

# Minimum phase angle to consider
min_phase = 3

In [5]:
# ZTF filters 1: g, 2: r
filters = {'1': 'g', '2': 'r'}
fink_colors = ['#15284F', '#F5622E']

In [15]:
data = pd.read_parquet(os.path.join(data_fink, 'data', 'ztf', 'sso_ZTF.parquet'))

In [16]:
data['SHG1G2_dSpin'] = np.sqrt( (data['SHG1G2_dalpha0']*np.cos(np.radians(data['SHG1G2_delta0'])))**2 + data['SHG1G2_ddelta0']**2 )


# # Remove solutions above 90 deg of latitude
cond = data.SHG1G2_delta0 > 90
data.loc[cond,'SHG1G2_delta0'] = 90
print(f'above 90: {len(data[cond])} ')

cond = data.SHG1G2_delta0 < -90
data.loc[cond,'SHG1G2_delta0'] = -90
print(f'below 90: {len(data[cond])} ')


above 90: 0 
below 90: 0 


In [17]:
cols = ['sso_number', 'sso_name', 'sso_class',
        'orbital_elements.semi_major_axis.value',
        'orbital_elements.eccentricity.value',
        'orbital_elements.inclination.value',
        'orbital_elements.node_longitude.value',
        'orbital_elements.perihelion_argument.value',
        'orbital_elements.mean_anomaly.value',
        'orbital_elements.mean_motion.value',
        'family.family_number', 'family.family_name',
        'proper_elements.proper_semi_major_axis.value',
        'proper_elements.proper_eccentricity.value',
        'proper_elements.proper_inclination.value',
        'proper_elements.proper_sine_inclination.value',
        'tisserand_parameters.Jupiter.value',
        'albedo.value',
        'absolute_magnitude.value', 
        'diameter.value', 
        'taxonomy.class', 'taxonomy.complex', 'taxonomy.waverange', 'taxonomy.scheme', 'taxonomy.technique',
        'colors.g-r.color.value', 'colors.g-r.color.error.min', 'colors.g-r.color.error.max',
        'colors.g-r.facility', 'colors.g-r.observer', 'colors.g-r.epoch',
        'colors.g-r.delta_time', 'colors.g-r.id_filter_1',
        'colors.g-r.id_filter_2', 'colors.g-r.phot_sys', 'colors.g-r.technique',
        'spins.1.obliquity', 'spins.1.RA0.value', 'spins.1.DEC0.value', 'spins.1.RA0.error.max', 'spins.1.DEC0.error.max',
        'spins.1.long.value', 'spins.1.lat.value', 'spins.1.technique',
        'spins.2.obliquity', 'spins.2.RA0.value', 'spins.2.DEC0.value', 'spins.2.RA0.error.max', 'spins.2.DEC0.error.max',
        'spins.2.long.value', 'spins.2.lat.value', 'spins.2.technique',
        'spins.3.obliquity', 'spins.3.RA0.value', 'spins.3.DEC0.value', 'spins.3.RA0.error.max', 'spins.3.DEC0.error.max',
        'spins.3.long.value', 'spins.3.lat.value', 'spins.3.technique',
        'spins.4.obliquity', 'spins.4.RA0.value', 'spins.4.DEC0.value', 'spins.4.RA0.error.max', 'spins.4.DEC0.error.max',
        'spins.4.long.value', 'spins.4.lat.value', 'spins.4.technique'
       ]        

bft = pd.read_parquet(bft_file, columns=cols)

In [18]:
damit = pd.read_csv( os.path.join(data_fink,'data','damit.csv' ) )
nam_num = rocks.identify(damit['name'])
damit['sso_number'] = [ nn[1] for nn in nam_num]
damit['sso_name'] = [ nn[0] for nn in nam_num]

damit = damit.drop_duplicates('sso_name')

In [19]:
data = data.merge( bft, left_on='name', right_on='sso_name', how='left' )

In [20]:
damit

,model_id,asteroid_id,lambda,beta,period,number,name,designation,reference_id,bibcode,R,R2,sso_number,sso_name
0,101,101,35,-12,7.813230,2.0,Pallas,NaN,106,2003icar..164..346t,0.910761,0.909266,2.0,Pallas
1,106,104,340,42,7.274471,6.0,Hebe,NaN,106,2003icar..164..346t,0.912084,0.874405,6.0,Hebe
2,110,106,335,-5,12.866670,8.0,Flora,NaN,106,2003icar..164..346t,0.910328,0.898977,8.0,Flora
3,111,107,180,22,5.079176,9.0,Metis,NaN,106,2003icar..164..346t,0.677687,0.654934,9.0,Metis
4,117,111,98,57,7.443224,19.0,Fortuna,NaN,106,2003icar..164..346t,0.789062,0.858640,19.0,Fortuna
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6227,6195,3566,182,-9,10.110700,83.0,Beatrix,NaN,662,2021a&a...654a..48h,0.817180,0.874277,83.0,Beatrix
6229,6197,3568,305,44,7.418710,945.0,Barcelona,NaN,662,2021a&a...654a..48h,0.774259,0.707804,945.0,Barcelona
6230,6198,3569,104,-43,16.438100,949.0,Hel,NaN,662,2021a&a...654a..48h,0.875412,0.930991,949.0,Hel
6231,6199,3570,219,-46,9.606520,971.0,Alsatia,NaN,662,2021a&a...654a..48h,0.865923,0.904425,971.0,Alsatia


In [21]:
data

,ssnamenr,HG_dG_g,HG_fit,HG_median_error_phot_1,HG_median_error_phot_2,HG_rms,HG_chi2red,HG_median_error_phot,HG_dH_g,HG_dG_r,...,spins.3.lat.value,spins.3.technique,spins.4.obliquity,spins.4.RA0.value,spins.4.DEC0.value,spins.4.RA0.error.max,spins.4.DEC0.error.max,spins.4.long.value,spins.4.lat.value,spins.4.technique
0,121056,0.175643,0.0,0.151332,0.123222,0.219825,4.811163,0.135677,0.145903,0.092701,...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
1,320925,0.103156,0.0,0.158014,0.131110,0.102925,0.886955,0.141857,0.104699,0.073903,...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
2,28493,0.154302,0.0,0.129726,0.091970,0.269923,8.031009,0.108044,0.106249,0.136220,...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
3,93190,0.071995,0.0,0.130041,0.101500,0.115412,1.087959,0.112057,0.048739,0.052689,...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
4,5136,0.139395,0.0,0.053467,0.057389,0.325366,48.614780,0.056247,0.151566,0.093985,...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120944,21345,0.068411,0.0,0.112095,0.115296,0.167127,3.180854,0.112441,0.080797,0.050744,...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
120945,31894,0.071089,0.0,0.085666,0.080628,0.179023,10.290506,0.082545,0.074112,0.055766,...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
120946,106592,0.054659,0.0,0.105252,0.092382,0.089848,1.120303,0.104463,0.046720,0.044332,...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
120947,56336,0.098171,0.0,0.138144,0.100554,0.238493,6.201401,0.116035,0.108521,0.093579,...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [23]:

xm = damit.merge(data, how='left', on='sso_name')
# len(xm)
# data['sso_name']

In [25]:
thres = 1e-3

# HG Parameeter
mask_HG_g = (data.HG_H_g.notna()) & (data.HG_G_g.notna())
mask_HG_r = (data.HG_H_r.notna()) & (data.HG_G_r.notna())
mask_HG_fit = (data.HG_fit == 0) & (data.HG_status >= 2)
mask_HG = mask_HG_g & mask_HG_r & mask_HG_fit

# HG1G2 parameters
mask_HG1G2_g = (
    (data.HG1G2_G1_g > thres)
    & (data.HG1G2_G2_g > thres)
    & ((1 - data.HG1G2_G1_g - data.HG1G2_G2_g) > thres)
)
mask_HG1G2_r = (
    (data.HG1G2_G1_r > thres)
    & (data.HG1G2_G2_r > thres)
    & ((1 - data.HG1G2_G1_r - data.HG1G2_G2_r) > thres)
)
mask_HG1G2_fit = (data.HG1G2_fit == 0) & (data.HG1G2_status >= 2)
mask_HG1G2 = mask_HG1G2_fit & mask_HG1G2_g & mask_HG1G2_r

# SHG1G2 ZTF
mask_SHG1G2_g = (
    (data.SHG1G2_G1_g > thres)
    & (data.SHG1G2_G2_g > thres)
    & ((1 - data.SHG1G2_G1_g - data.SHG1G2_G2_g) > thres)
)
mask_SHG1G2_r = (
    (data.SHG1G2_G1_r > thres)
    & (data.SHG1G2_G2_r > thres)
    & ((1 - data.SHG1G2_G1_r - data.SHG1G2_G2_r) > thres)
)
mask_SHG1G2_ZTF = mask_SHG1G2_g & mask_SHG1G2_r
mask_SHG1G2_fit = (data.SHG1G2_fit == 0) & (data.SHG1G2_status >= 2)
mask_SHG1G2 = mask_SHG1G2_fit & mask_SHG1G2_ZTF

# Oblateness
mask_R = (data.SHG1G2_R>0.3)

# Spin solution suspicous: RA=={0,180,360}, DEC==0
maskSpin = (
    (data.SHG1G2_alpha0.notna()) 
    & (data.SHG1G2_delta0.notna()) 
    & (data.SHG1G2_alpha0 > thres)
    & (np.abs(360 - data.SHG1G2_alpha0) > thres)
    & (np.abs(data.SHG1G2_alpha0 - 180) > thres)
    & (np.abs(data.SHG1G2_delta0) > thres)
)

# FINK Sample
maskFINK = mask_SHG1G2 & mask_R & maskSpin

# Phase coverage
maskPhase = data.min_phase < 2.5

# Global mask
mask = mask_HG1G2 & mask_SHG1G2


print(f" All data       : {len(data):6d}  ({100:>6.2f}%)")
print()
print(
    f"  Mask HG g      : {len(data[mask_HG_g]):6d}  ({100.*len(data[mask_HG_g])/len(data):>6.2f}%)"
)
print(
    f"  Mask HG r      : {len(data[mask_HG_r]):6d}  ({100.*len(data[mask_HG_r])/len(data):>6.2f}%)"
)
print(
    f"  Mask HG g+r    : {len(data[mask_HG]):6d}  ({100.*len(data[mask_HG])/len(data):>6.2f}%)"
)
print()
print(
    f"  Mask HG1G2 g   : {len(data[mask_HG1G2_g]):6d}  ({100.*len(data[mask_HG1G2_g])/len(data):>6.2f}%)"
)
print(
    f"  Mask HG1G2 r   : {len(data[mask_HG1G2_r]):6d}  ({100.*len(data[mask_HG1G2_r])/len(data):>6.2f}%)"
)
print(
    f"  Mask HG1G2 g+r : {len(data[mask_HG1G2]):6d}  ({100.*len(data[mask_HG1G2])/len(data):>6.2f}%)"
)
print()
print(
    f"  Mask SHG1G2 g  : {len(data[mask_SHG1G2_g]):6d}  ({100.*len(data[mask_SHG1G2_g])/len(data):>6.2f}%)"
)
print(
    f"  Mask SHG1G2 r  : {len(data[mask_SHG1G2_r]):6d}  ({100.*len(data[mask_SHG1G2_r])/len(data):>6.2f}%)"
)
print(
    f"  Mask SHG1G2 g+r: {len(data[mask_SHG1G2]):6d}  ({100.*len(data[mask_SHG1G2])/len(data):>6.2f}%)"
)
print()
print(
    f"  Mask Oblateness: {len(data[mask_R]):6d}  ({100.*len(data[mask_R])/len(data):>6.2f}%)"
)
print(
    f"  Mask Spin      : {len(data[maskSpin]):6d}  ({100.*len(data[maskSpin])/len(data):>6.2f}%)"
)
print()
print(
    f"  Mask FINK      : {len(data[maskFINK]):6d}  ({100.*len(data[maskFINK])/len(data):>6.2f}%)"
)
print()
print(
    f"  Mask (both)    : {len(data[mask]):6d}  ({100.*len(data[mask])/len(data):>6.2f}%)"
)
for minphase in [2, 3, 4, 5]:
    maskPhase = data.min_phase < minphase
    print(
        f"  Mask phase {minphase}  : {len(data[maskPhase]):6d}  ({100.*len(data[maskPhase])/len(data):>6.2f}%)"
    )


# len(data), len(data[mask]), len(data[mask_HG1G2]), len(data[mask_HG1G2sp]), len(data[mask_SHG1G2_g]), len(data[mask_SHG1G2_r]), len(data[mask_SHG1G2])

 All data       : 120949  (100.00%)

  Mask HG g      : 120651  ( 99.75%)
  Mask HG r      : 120653  ( 99.76%)
  Mask HG g+r    : 118312  ( 97.82%)

  Mask HG1G2 g   :  62804  ( 51.93%)
  Mask HG1G2 r   :  67827  ( 56.08%)
  Mask HG1G2 g+r :  46254  ( 38.24%)

  Mask SHG1G2 g  :  78530  ( 64.93%)
  Mask SHG1G2 r  :  83479  ( 69.02%)
  Mask SHG1G2 g+r:  63099  ( 52.17%)

  Mask Oblateness:  90168  ( 74.55%)
  Mask Spin      : 117078  ( 96.80%)

  Mask FINK      :  49235  ( 40.71%)

  Mask (both)    :  40117  ( 33.17%)
  Mask phase 2  :  61691  ( 51.01%)
  Mask phase 3  :  84622  ( 69.97%)
  Mask phase 4  :  99015  ( 81.87%)
  Mask phase 5  : 107296  ( 88.71%)


# Stat observations

In [26]:
q_obs = data['n_obs'].quantile([0.25,0.50,0.75])
q_day = data['n_days'].quantile([0.25,0.50,0.75])

print('Observation            ${:.0f}_{{{:-.0f}}}^{{{:+.0f}}}$'.format( q_obs[0.50], q_obs[0.25]-q_obs[0.50], q_obs[0.75]-q_obs[0.50] ))
print('Days                   ${:.0f}_{{{:-.0f}}}^{{{:+.0f}}}$'.format( q_day[0.50], q_day[0.25]-q_day[0.50], q_day[0.75]-q_day[0.50] ))

q_obs = data.loc[maskFINK,'n_obs'].quantile([0.25,0.50,0.75])
q_day = data.loc[maskFINK,'n_days'].quantile([0.25,0.50,0.75])

print('Observation (maskFINK) ${:.0f}_{{{:-.0f}}}^{{{:+.0f}}}$'.format( q_obs[0.50], q_obs[0.25]-q_obs[0.50], q_obs[0.75]-q_obs[0.50] ))
print('Days        (maskFINK) ${:.0f}_{{{:-.0f}}}^{{{:+.0f}}}$'.format( q_day[0.50], q_day[0.25]-q_day[0.50], q_day[0.75]-q_day[0.50] ))



Observation            $92_{-26}^{+45}$
Days                   $92_{-26}^{+45}$
Observation (maskFINK) $103_{-31}^{+48}$
Days        (maskFINK) $103_{-31}^{+48}$


# Compare with literature

In [27]:
cond = (data['spins.1.RA0.value'].notna()) & maskFINK
data.loc[cond,'spins.1.technique'].value_counts()

spins.1.technique
LCI       3780
LC+TPM     213
LC          29
LC-TPM      17
ADAM        11
TE          10
LC+Occ       5
A-M          3
Radar        3
SAGE         1
SPACE        1
Name: count, dtype: int64

In [28]:
# Compute angular distances
spin_version = np.array(['A','B'])

for ks in ['1','2','3','4']:
    data[f'spins.{ks}.distance'] = np.nan
    cond = (~data[f'spins.{ks}.RA0.value'].isna())
    
    #------------------------------------------------------------------------------------------
    # A: Original spins
    data.loc[cond,f'spins.{ks}.distance.A'] = data.loc[cond,[f'spins.{ks}.RA0.value',f'spins.{ks}.DEC0.value', 'SHG1G2_alpha0','SHG1G2_delta0']].apply(
        lambda x: np.degrees(angular_separation( np.radians(x[0]), np.radians(x[1]), np.radians(x[2]), np.radians(x[3]) )) , axis=1
    )
    data.loc[cond,f'spins.{ks}.delta_RA.A'] = (data.loc[cond,f'spins.{ks}.RA0.value']-data.loc[cond,'SHG1G2_alpha0'] ) * np.cos(np.radians(data.loc[cond,f'spins.{ks}.DEC0.value']))
    data.loc[cond,f'spins.{ks}.delta_DEC.A'] = (data.loc[cond,f'spins.{ks}.DEC0.value']-data.loc[cond,'SHG1G2_delta0'] )
    data.loc[cond,f'spins.{ks}.RA.A'] = data.loc[cond,f'spins.{ks}.RA0.value']
    data.loc[cond,f'spins.{ks}.DEC.A'] = data.loc[cond,f'spins.{ks}.DEC0.value']

    #------------------------------------------------------------------------------------------
    # B: RA-180 & Flip DEC
    data.loc[cond,f'spins.{ks}.distance.B'] = data.loc[cond,[f'spins.{ks}.RA0.value',f'spins.{ks}.DEC0.value', 'SHG1G2_alpha0','SHG1G2_delta0']].apply(
        lambda x: np.degrees(angular_separation( np.radians( (x[0]+180) % 360 ), np.radians(-x[1]), np.radians(x[2]), np.radians(x[3]) )) , axis=1
    )
    data.loc[cond,f'spins.{ks}.delta_RA.B'] = ( ((data.loc[cond,f'spins.{ks}.RA0.value']+180.0) % 360 )-data.loc[cond,'SHG1G2_alpha0'] ) * np.cos(np.radians(-data.loc[cond,f'spins.{ks}.DEC0.value']))
    data.loc[cond,f'spins.{ks}.delta_DEC.B'] = (-data.loc[cond,f'spins.{ks}.DEC0.value']-data.loc[cond,'SHG1G2_delta0'] )
    data.loc[cond,f'spins.{ks}.RA.B'] = ((data.loc[cond,f'spins.{ks}.RA0.value']+180.0) % 360 )
    data.loc[cond,f'spins.{ks}.DEC.B'] = -data.loc[cond,f'spins.{ks}.DEC0.value']
    
    #------------------------------------------------------------------------------------------
    # Find minimum distance and keep track of which spin version it is
    data.loc[cond,f'spins.{ks}.closest'] = data.loc[cond, [f'spins.{ks}.distance.A', 
                                                           f'spins.{ks}.distance.B']].apply( lambda x: int(np.argmin(x)), axis=1 )
    data = data.astype({f'spins.{ks}.closest':'Int64'})

    data.loc[cond,f'spins.{ks}.distance'] = data.loc[cond, [f'spins.{ks}.distance.A', 
                                                            f'spins.{ks}.distance.B']].apply( lambda x: np.min(x), axis=1 )
    data.loc[cond,f'spins.{ks}.version'] = data.loc[cond, [f'spins.{ks}.distance.A', 
                                                           f'spins.{ks}.distance.B']].apply( lambda x: spin_version[np.argmin(x)], axis=1 )

    #------------------------------------------------------------------------------------------
    # Find minimum distance and keep track of which spin version it is
    data.loc[cond,f'spins.{ks}.delta_RA'] = data.loc[cond, [f'spins.{ks}.delta_RA.A',
                                                            f'spins.{ks}.delta_RA.B', 
                                                            f'spins.{ks}.closest']].apply( lambda x: x[int(x[2])], axis=1 )
    data.loc[cond,f'spins.{ks}.delta_DEC'] = data.loc[cond, [f'spins.{ks}.delta_DEC.A',
                                                             f'spins.{ks}.delta_DEC.B', 
                                                             f'spins.{ks}.closest']].apply( lambda x: x[int(x[2])], axis=1 )
    data.loc[cond,f'spins.{ks}.RA'] = data.loc[cond, [f'spins.{ks}.RA.A',
                                                      f'spins.{ks}.RA.B', 
                                                      f'spins.{ks}.closest']].apply( lambda x: x[int(x[2])], axis=1 )
    data.loc[cond,f'spins.{ks}.DEC'] = data.loc[cond, [f'spins.{ks}.RA.A',
                                                       f'spins.{ks}.RA.B', 
                                                       f'spins.{ks}.closest']].apply( lambda x: x[int(x[2])], axis=1 )

    
# Select the closest angularly
cond = ~data[f'spins.1.distance'].isna()
data.loc[cond,'spins.closest'] = data.loc[cond,['spins.1.distance','spins.2.distance','spins.3.distance','spins.4.distance']].apply( lambda x: int(np.argmin(x)), axis=1 )
data = data.astype({'spins.closest':'Int64'})

data.loc[cond,'spins.distance'] = data.loc[cond,['spins.1.distance','spins.2.distance','spins.3.distance','spins.4.distance']].apply( lambda x: np.min(x), axis=1 )
data.loc[cond,'spins.delta_RA'] = data.loc[cond, [f'spins.1.delta_RA',
                                                  f'spins.2.delta_RA',
                                                  f'spins.3.delta_RA',
                                                  f'spins.4.delta_RA', 
                                                  f'spins.closest']].apply( lambda x: x[int(x[4])], axis=1 )
data.loc[cond,'spins.delta_DEC'] = data.loc[cond, [f'spins.1.delta_DEC',
                                                   f'spins.2.delta_DEC',
                                                   f'spins.3.delta_DEC',
                                                   f'spins.4.delta_DEC', 
                                                   f'spins.closest']].apply( lambda x: x[int(x[4])], axis=1 )

data.loc[cond,'spins.err_RA'] = data.loc[cond, [f'spins.1.RA0.error.max',
                                                f'spins.2.RA0.error.max',
                                                f'spins.3.RA0.error.max',
                                                f'spins.4.RA0.error.max', 
                                                f'spins.closest']].apply( lambda x: x[int(x[4])], axis=1 )
data.loc[cond,'spins.err_DEC'] = data.loc[cond, [f'spins.1.DEC0.error.max',
                                                 f'spins.2.DEC0.error.max',
                                                 f'spins.3.DEC0.error.max',
                                                 f'spins.4.DEC0.error.max', 
                                                 f'spins.closest']].apply( lambda x: x[int(x[4])], axis=1 )

data.loc[cond,'spins.RA'] = data.loc[cond, [f'spins.1.RA',
                                            f'spins.2.RA',
                                            f'spins.3.RA',
                                            f'spins.4.RA', 
                                            f'spins.closest']].apply( lambda x: x[int(x[4])], axis=1 )
data.loc[cond,'spins.DEC'] = data.loc[cond, [f'spins.1.DEC',
                                             f'spins.2.DEC',
                                             f'spins.3.DEC',
                                             f'spins.4.DEC', 
                                             f'spins.closest']].apply( lambda x: x[int(x[4])], axis=1 )


/tmp/ipykernel_56485/3374077660.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lambda x: np.degrees(angular_separation( np.radians(x[0]), np.radians(x[1]), np.radians(x[2]), np.radians(x[3]) )) , axis=1
/tmp/ipykernel_56485/3374077660.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lambda x: np.degrees(angular_separation( np.radians( (x[0]+180) % 360 ), np.radians(-x[1]), np.radians(x[2]), np.radians(x[3]) )) , axis=1
/home/bcarry/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: The behavior of Series.argmax/argmin with skipna=False and NAs, or with all-NAs is deprecated. 

In [29]:
# Set errors to 15 deg if missing
for c in ['spins.RA','spins.DEC','spins.err_RA', 'spins.err_DEC']:
    missing = data[c].isna()
    data.loc[missing,c] = 15.

In [30]:
# Checking for missing values
for c in ['spins.RA','spins.DEC','spins.err_RA', 'spins.err_DEC']:
    missing = data[c].isna()
    print(c, len(data[missing]))
    data[c] = data[c].astype(float)
len(data)

spins.RA 0
spins.DEC 0
spins.err_RA 0
spins.err_DEC 0


120949

In [31]:
data.loc[cond,'spins.dSpin'] = np.sqrt( (data.loc[cond,'spins.err_RA']*np.cos(np.radians(data.loc[cond,'spins.DEC'])))**2 + data.loc[cond,'spins.err_DEC']**2 )


/tmp/ipykernel_56485/1164014787.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data.loc[cond,'spins.dSpin'] = np.sqrt( (data.loc[cond,'spins.err_RA']*np.cos(np.radians(data.loc[cond,'spins.DEC'])))**2 + data.loc[cond,'spins.err_DEC']**2 )


# Main figure for article

In [ ]:
# --------------------------------------------------------------------------------
fig, ax = plt.subplots(
    1,
    2,
    figsize=fs.figsize(0.5),
    gridspec_kw={
        "hspace": 0.02,
        "wspace": 0.02,
        "top": 0.98,
        "bottom": 0.13,
        "left": 0.06,
        "right": 0.98,
    },
)

# --------------------------------------------------------------------------------
# Distance in degrees
r = [0, 90]
b = 18

ax[0].hist(
    data.loc[maskFINK, "spins.distance"],
    range=r,
    bins=b,
    density=True,
    label="Select",
    alpha=0.8,
)
ax[0].set_xlabel("Angular distance / deg")

ax0 = ax[0].twinx()
ax0.hist(
    data.loc[maskFINK, "spins.distance"],
    range=r,
    bins=b,
    density=True,
    cumulative=True,
    histtype="step",
    color="darkgrey",
)
ax[0].set_xlim(r)


# --------------------------------------------------------------------------------
# Distance in unit of uncertainty
r = [0, 5]
b = 25

x = data["spins.distance"] / np.sqrt(
    data["SHG1G2_dSpin"] ** 2 + data["spins.dSpin"] ** 2
)
ax[1].hist(x[maskFINK], range=r, bins=b, density=True, alpha=0.8)
ax[1].set_xlabel("Angular distance / $\\sigma$")

ax1 = ax[1].twinx()
ax1.hist(
    x[maskFINK],
    range=r,
    bins=b,
    density=True,
    cumulative=True,
    histtype="step",
    color="darkgrey",
)
ax[1].set_xlim(r)

# --------------------------------------------------------------------------------
# Axes
ax[0].set_ylim(0, 0.025)
ax[1].set_ylim(0, 1)
for a in ax:
    a.set_yticklabels("")
for a in [ax0, ax1]:
    a.set_yticklabels("")

ax[0].set_ylabel('Density count')

# --------------------------------------------------------------------------------
fig.savefig(
    os.path.join(data_fink, "gfx", "article", "spin_distance.png"),
    facecolor="white",
)

fig.savefig(
    os.path.join(data_fink, "gfx", "article", "spin_distance.pgf"),
)
plt.close()

# Misc for test

In [ ]:
data.loc[maskFINK, "spins.distance"].count(), len(data.loc[maskFINK, "spins.distance"])

In [ ]:
cond = (data['spins.1.RA0.value'].notna()) & maskFINK
techs = data.loc[cond,'spins.1.technique'].value_counts()
data.loc[cond,'spins.1.technique'].value_counts()

In [ ]:
techs.sum()

In [ ]:
cond = (
    (data["spins.1.RA0.value"].notna())
    & maskFINK
    & (data["spins.1.technique"] == "ADAM")
)
data.loc[cond, ['sso_number', 'sso_name', "spins.1.technique"] ]

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(18,5))

r = [0,10]
b = 50

x = data['spins.distance'] / np.sqrt( data['SHG1G2_dSpin']**2 + data['spins.dSpin']**2 )
ax[0].hist( x, range=r, bins=b, density=True, label='All' )
ax[0].hist( x[maskFINK], range=r, bins=b, density=True, label='Select', color='grey', alpha=0.8 )
ax[0].legend(loc='center right')
ax[0].set_xlabel('Distances / deg')
ax0 = ax[0].twinx()
ax0.hist( x[maskFINK], range=r, bins=b, density=True, cumulative=True, histtype='step', color='black' )
ax[0].set_xlim(r)

x = data['spins.delta_RA'] / data['SHG1G2_dalpha0']
ax[1].hist( x, range=r, bins=b, density=True, label='All' )
ax[1].hist( x[maskFINK], range=r, bins=b, density=True, label='Select', color='grey', alpha=0.8 )
ax[1].legend(loc='center right')
ax[1].set_xlabel('$\Delta$ RA cos(DEC)')
ax1 = ax[1].twinx()
ax1.hist( x[maskFINK], range=r, bins=b, density=True, cumulative=True, histtype='step', color='black' )
ax[1].set_xlim(r)

x = data['spins.delta_DEC'] / data['SHG1G2_ddelta0']
ax[2].hist( x, range=r, bins=b, density=True, label='All' )
ax[2].hist( x[maskFINK], range=r, bins=b, density=True, label='Select', color='grey', alpha=0.8 )
ax[2].legend(loc='center right')
ax[2].set_xlabel('$\Delta$ DEC')
ax2 = ax[2].twinx()
ax2.hist( x[maskFINK], range=r, bins=b, density=True, cumulative=True, histtype='step', color='black' )
ax[2].set_xlim(r)

fig.savefig( os.path.join(data_fink, 'gfx', 'valid', 'sky_distances_normed.png'), facecolor='white')

In [ ]:
fig, ax = plt.subplots( 1, 2, figsize=(12,4))

r = [ [0,90],[0,10] ]
b = [36,20]

x = data['spins.distance'] 
y = data['spins.distance'] / data['SHG1G2_dSpin']
ax[0].hist2d( x, y, label='All', range=r, bins=b, cmap='Blues', norm='log', density=True )
ax[1].hist2d( x[maskFINK], y[maskFINK], label='Select', range=r, bins=b, cmap='Blues', norm='log', density=True)

# ax[0].legend(loc='center right')
for a in ax:
    a.set_xlabel('Distances / deg')
    a.set_ylabel('Distances / sigma')

fig.savefig( os.path.join(data_fink, 'gfx', 'valid', 'sky_distances_vs_sigma.png'), facecolor='white')

In [ ]:
fig, ax = plt.subplots(
    1,
    3,
    sharey=True,
    figsize=(18,5)
)

# --------------------------------------------------------------------------------
x = data["min_phase"]
y = data["spins.distance"]
# ax[0].scatter( x,y, label='All' )
ax[0].scatter(x[maskFINK], y[maskFINK], marker=".", s=4, label="Select")
ax[0].legend()
ax[0].set_xlabel("Min phase")
ax[0].set_ylabel("Distances / deg")
ax[0].set_xlim(0, 10)

# --------------------------------------------------------------------------------
x = data["SHG1G2_rms"]
y = data["spins.distance"]
# ax[1].scatter( x,y, label='All' )
ax[1].scatter(x[maskFINK], y[maskFINK], marker=".", s=4, label="Select")
ax[1].legend()
ax[1].set_xlabel("RMS")
# ax[1].set_xscale("log")

# --------------------------------------------------------------------------------
x = np.degrees(
    np.arccos(data["SHG1G2_min_cos_lambda"]) - np.arccos(data["SHG1G2_max_cos_lambda"])
)
y = data["spins.distance"]
ax[2].scatter( x[maskFINK], y[maskFINK], c=data.loc[maskFINK,'SHG1G2_R'], label='Select' )
ax[2].legend()
ax[2].set_xlabel("Amplitude Lambda")
# ax[1].set_ylabel("Distances / deg")

fig.savefig(
    os.path.join(data_fink, "gfx", "valid", "sky_distances_vs_obs.png"),
    facecolor="white",
    dpi=180,
)

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

x = np.degrees(np.arccos(data['SHG1G2_min_cos_lambda']) - np.arccos(data['SHG1G2_max_cos_lambda']))
y = data['spins.distance']
im = ax.scatter( x[maskFINK], y[maskFINK], c=data.loc[maskFINK,'SHG1G2_R'], label='Select' )
ax.legend()
ax.set_xlabel('Amplitude Lambda')
ax.set_ylabel('Distances / deg')

fig.colorbar(im)

fig.savefig( os.path.join(data_fink, 'gfx', 'valid', 'sky_distances_vs_Lambda.png'), facecolor='white')


In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

x = np.degrees(np.arccos(data['SHG1G2_min_cos_lambda']) - np.arccos(data['SHG1G2_max_cos_lambda']))
y = data['SHG1G2_R']

# im = ax.scatter( x[maskFINK], y[maskFINK], marker='.', s=1, label='Select' )
ax.hist2d( x[maskFINK], y[maskFINK], 
          range= [ [0,90], [0.3,1]], bins=[18,14] )



ax.set_xlabel('Amplitude Lambda')
ax.set_ylabel('R')

fig.savefig( os.path.join(data_fink, 'gfx', 'valid', 'Lambda_vs_R.png'), facecolor='white')


In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(18,5))

cond = data.ssnamenr==data.ssnamenr

ax[0].scatter( data.loc[cond,'n_days'], data.loc[cond,'spins.distance'], label='All' )
ax[0].scatter( data.loc[maskFINK,'n_days'], data.loc[maskFINK,'spins.distance'], label='Select' )
ax[0].legend()
ax[0].set_xlabel('N days')
ax[0].set_ylabel('Distances / deg')

ax[1].scatter( data.loc[cond,'SHG1G2_chi2red'], data.loc[cond,'spins.distance'], label='All' )
ax[1].scatter( data.loc[maskFINK,'SHG1G2_chi2red'], data.loc[maskFINK,'spins.distance'], label='Select' )
ax[1].legend()
ax[1].set_xlabel('chi2 red')
ax[1].set_xscale('log')
ax[1].set_ylabel('Distances / deg')

ax[2].scatter( data.loc[cond,'absolute_magnitude.value'], data.loc[cond,'spins.distance'], label='All' )
ax[2].scatter( data.loc[maskFINK,'absolute_magnitude.value'], data.loc[maskFINK,'spins.distance'], label='Select' )
ax[2].legend()
ax[2].set_xlabel('abs mag')
# ax[2].set_xscale('log')
ax[2].set_ylabel('Distances / deg')
fig.savefig( os.path.join(data_fink, 'gfx', 'valid', 'sky_distances_vs_Lambda.png'), facecolor='white')


In [ ]:
r = [0,20]
b = 200 

fig, ax = plt.subplots()
ax.hist(data['SHG1G2_chi2red'], range=r, bins=b, alpha=0.5, density=True )
# ax.hist(data.loc[maskFINK,'SHG1G2_chi2red'], range=r, bins=b, alpha=0.5, density=True )

cond = data['spins.1.RA0.value'].notna()
ax.hist(data.loc[cond,'SHG1G2_chi2red'], range=r, bins=b, alpha=0.5, density=True )

cond = data['spins.1.RA0.value'].notna() & maskFINK
ax.hist(data.loc[cond,'SHG1G2_chi2red'], range=r, bins=b, alpha=0.5, density=True )
# ax.xlim(0,10)
ax.set_xlabel('chi2')

len(data[maskFINK]), len(data[cond])

data.loc[cond, ['ssnamenr','SHG1G2_rms','SHG1G2_chi2red']].sort_values(by='SHG1G2_rms')
# data[cond]

- fishy: the object with spins have shitty chi2red!
- Explanation: there is bias for high amplitude for LC-derived spins -> bad for HG1G2hybrid (LC not taken into account)

In [ ]:
fig, ax = plt.subplots()

r = [0,20]
b = 200 


ax.scatter(xm['SHG1G2_R']

In [ ]:
r = [0,0.5]
b = 100 
# plt.hist(data['SHG1G2_rms'], range=r, bins=b, alpha=0.5, density=True )
plt.hist(data.loc[maskFINK,'SHG1G2_rms'], range=r, bins=b, alpha=0.5, density=True )

cond = data['spins.1.RA0.value'].notna()
plt.hist(data.loc[cond,'SHG1G2_rms'], range=r, bins=b, alpha=0.5, density=True )
# plt.xlim(0,10)
plt.xlabel('chi2')

len(data[maskFINK]), len(data[cond])

In [ ]:
fig, ax = plt.subplots()

ax.hist( data.loc[cond,'diameter.value'], histtype='step', bins=np.linspace(1,200,num=200), density=True, label='lit')
ax.hist( data.loc[maskFINK,'diameter.value'], histtype='step', bins=np.linspace(1,200,num=200), density=True, label='fink' )

ax.set_xscale('log')
ax.legend()

In [ ]:
cond = maskFINK
lim_spin = 30
spin_v = '1'
cond = cond * (data[f'spins.{spin_v}.distance'] < lim_spin)
data.loc[cond,f'spins.{spin_v}.technique'].value_counts()

# plt.hist( data.loc[cond,'spins.1.distance'] )

In [ ]:
cond = maskFINK
data[cond].sort_values(by='sso_number').head(20)

In [ ]:
cond = maskFINK
showcols = ['number','name', #'minphase','maxphase','n_days',
        'SHG1G2_alpha0','SHG1G2_delta0',
        'spins.1.RA0.value','spins.1.DEC0.value', 'spins.1.version',
        'spins.2.RA0.value','spins.2.DEC0.value', 'spins.2.version',
        'spins.1.distance', 'spins.2.distance']
data.loc[cond,showcols].sort_values(by='number').head(20)

In [ ]:
data.columns[:80]

# Obliquity

In [ ]:
# Obliquity of the spin
data['lon_orbit'] = data['orbital_elements.node_longitude.value'] - 90
data['lat_orbit'] = 90. - data['orbital_elements.inclination.value']
data['obliquity'] = data[['lon','lat', 'lon_orbit','lat_orbit']].apply(
    lambda x: np.degrees(angular_separation( np.radians(x[0]), np.radians(x[1]), np.radians(x[2]), np.radians(x[3]) )) , axis=1
)


In [ ]:
fig, ax = plt.subplots(figsize=(12,8), 
                      gridspec_kw={'right':0.975, 
                                  'top':0.95, 
                                  'bottom':0.15})

cond = data.ssnamenr==data.ssnamenr
cond = maskFINK
ax.scatter( data.loc[cond,'diameter.value'],
            data.loc[cond,'obliquity'], 
            marker='.',
            # color=colors[0], 
            # s=2, 
            alpha=0.15 )

ax.set_xscale('log')
ax.set_xlim(1,300)
ax.set_ylim(0,180)
ax.set_xlabel('Diameter / km')
ax.set_ylabel('Obliquity / deg')


ax.set_xticks([1,10,100])
ax.set_xticklabels(['1','10','100'])
ax.set_yticks([0,30,60,90,120,150,180])

fig.savefig( os.path.join(data_fink, 'gfx', 'valid', 'obliquity_vs_diameter.png'), facecolor='white')

In [ ]:
cond = maskFINK & (data['diameter.value']<10)
fig, ax = plt.subplots()

r = [0,180]
b = 37
ax.hist( data.loc[cond,'obliquity'], bins=b, range=r )

In [ ]:
np.sqrt(2*np.pi / (5*3600)), np.sqrt(2*np.pi / (5*365.24*86400))

In [ ]:
nu = [1e-8, 1e-4]

ti = 200
emissivity = 0.9
sigma_b = 5.670374419e-8
S_sun = 1360
beaming = 0.9
delta = 3.
pV = 0.15
A = (0.29 + 0.684*0.15) * pV

theta = ti*np.sqrt(nu) * ((beaming * delta**2 )**(3/4)) * ( (sigma_b*emissivity)**(-1/4) ) * ( ((1-A)*S_sun)**(-3/4) )

W = -0.5* theta / (1 + theta + 0.5*theta**2)

Wn = W[0]
Ww = W[1]

print(W)

In [ ]:
data.loc[maskFINK,'family.family_name'].value_counts().head(30)

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))

f = 'Astraea'

cond = bft['family.family_name']==f
ax.scatter( bft.loc[cond,'proper_elements.proper_semi_major_axis.value'], bft.loc[cond,'absolute_magnitude.value'], color='grey', alpha=0.2, marker='.' )

                    
cond = maskFINK & (data['family.family_name']==f)
im = ax.scatter( data.loc[cond,'proper_elements.proper_semi_major_axis.value'],
            data.loc[cond,'absolute_magnitude.value'],
            c=np.abs(data.loc[cond,'obliquity']-90),
            vmin=0, vmax=90, alpha=1.0, marker='o', cmap='Oranges' )

ax.set_xlabel('Proper semi-major axis / au')
ax.set_ylabel('Absolute magnitude H')
fig.colorbar(im)

# Oblateness

In [ ]:
np.mean(damit.R), np.mean(damit.R2)

In [ ]:
# fig, ax = plt.subplots(figsize=fs.figsize(0.5), 
fig, ax = plt.subplots(figsize=(10,6), 
                      gridspec_kw={'right':0.975, 
                                  'top':0.95, 
                                  'bottom':0.15})

r = [0,1]
b = 50

colors = ['#15284F', '#F5622E']


# ax.hist( damit.R, range=r, bins=b, 
#         color='lightgray', 
#         density=True, alpha=0.5, label=f'DAMIT ({len(damit):,d})' )

ax.hist( damit.R2, range=r, bins=b, 
        color='slategray', 
        density=True, alpha=0.5, label=f'DAMIT ({len(damit):,d})' )
# ax.hist( data.loc[maskFINK,'SHG1G2_R'], range=r, bins=b, density=True, alpha=0.5, label=f'FINK ({len(data[maskFINK]):,d})' )

cond = (data.SHG1G2_err_R<0.9) & maskFINK
ax.hist( data.loc[cond,'SHG1G2_R'], range=r, bins=b, 
         color=colors[0], 
        density=True, alpha=0.5, label=f'FINK ({len(data[maskFINK]):,d})' )

ax.set_ylim(0,4)
ax.set_xlabel('Oblateness R')
ax.set_ylabel('Count density')
ax.legend(loc='upper left')

# fig.savefig(f'{data_fink}plots/R_and_damit.png', facecolor='white', dpi=150)


In [ ]:
# fig, ax = plt.subplots(figsize=fs.figsize(0.5), 
fig, ax = plt.subplots(1, 2, figsize=(12,5), 
                       gridspec_kw={'right':0.975, 
                                  'top':0.95, 
                                  'bottom':0.15})

r = [0,1]
b = 50

colors = ['#15284F', '#F5622E']

x = damit.merge(data, left_on='number', right_on='sso_number')

# ax.scatter( x.R, x.SHG1G2_R ) 
ax[0].hist2d( x.R, x.SHG1G2_R, range=[[0.3,1],[0.3,1]], bins=14) 
xx = np.linspace(0.3,1,num=10)
ax[1].plot(xx,xx, color='white')

ax[1].hist( (x.R-x.SHG1G2_R)/x.SHG1G2_err_R, range=[-5,5], bins=20 )



# ax.set_ylim(0,4)
ax[0].set_xlabel('FINK R')
ax[0].set_ylabel('Count density')

ax[0].set_xlabel('(FINK R - DAMIT R) / dR')

# fig.savefig(f'{data_fink}plots/R_and_damit.png', facecolor='white', dpi=150)


In [ ]:
# damit.sort_values(by='R').head(40)
cond = maskFINK & (data.SHG1G2_R>0.18)
data[cond].sort_values(by='SHG1G2_R').head(40)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(18,5))

r = [0,1]
b = 50


cond = maskFINK

ax[0].scatter( data.loc[cond,'SHG1G2_G1_r'], data.loc[cond,'SHG1G2_R'], marker='.', alpha=0.15 )
ax[1].scatter( data.loc[cond,'SHG1G2_G2_r'], data.loc[cond,'SHG1G2_R'], marker='.', alpha=0.15 )
ax[2].scatter( data.loc[cond,'SHG1G2_G1_r']+data.loc[cond,'SHG1G2_G2_r'], data.loc[cond,'SHG1G2_R'], marker='.', alpha=0.15 )

ax[0].set_xlabel('G1')
ax[1].set_xlabel('G2')
ax[2].set_xlabel('G1 + G2')
ax[0].set_ylabel('R')

# ax.legend(loc='upper left')